In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [47]:
crowd_train = load_file('./data/train.csv/train.csv', None)
crowd_test = load_file('./data/test.csv/test.csv', None)

In [48]:
target = crowd_train.median_relevance.values

In [80]:
train_index, test_index = ssSplit(target, train_size=8000, random_state=44)

In [81]:
X = tweak_text(crowd_train.iloc[train_index])
Xv = tweak_text(crowd_train.iloc[test_index])
# X = tweak_text(crowd_train)
# Xv = tweak_text(crowd_test)

In [51]:
# y = target[train_index]
# yv = target[test_index]
y = target

In [52]:
Xfitted, tfv = TFIDF(X, None)

In [53]:
svm_non_linear, svd, scl = build_non_linear_model(Xfitted, y)

In [54]:
options = {
    'tfv': tfv,
    'svd': svd,
    'scl': scl
}

preds = make_predictions(svm_non_linear, options, Xv)

In [55]:
make_submission(crowd_test.id.values.astype(int), preds, '300features_svm.csv')

In [11]:
#print 'Kappa score on validation set %0.4f ' %(quadratic_weighted_kappa(yv, preds))

Kappa score on validation set 0.6150 


In [12]:
#from sklearn.metrics import confusion_matrix

In [13]:
#confusion_matrix(yv, preds)

array([[ 46,  15,   2,  14],
       [ 19,  53,  23,  53],
       [  6,  23,  33, 112],
       [  8,  20,  35, 554]])

In [14]:
# Analyze the mistakes of this model
mistakes_idx = [test_index[i] for i in range(len(preds)) if yv[i] != preds[i]]
incorrect_preds = [preds[i] for i in range(len(preds)) if yv[i] != preds[i]]

In [15]:
mistakes = crowd_train.iloc[mistakes_idx]

In [16]:
mistakes['pred'] = incorrect_preds

In [17]:
mistakes.shape

(330, 7)

In [18]:
mistakes.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,pred
8227,26520,thomas the train,Fisher-Price Power Wheels Thomas the Train 6-V...,Extra value! Get two more feet of track than t...,3,0.471,4
5695,18316,wii microphone,Tomee Chat Headset with Microphone for Wii U; ...,The Tomee Gaming Chat Headset for the Wii U al...,4,0.471,2
1058,3414,dr who lanyard,"PROTO JLANHATRD Hard Hat Lanyard, , 23 in, Red",,2,0.894,1
6449,20789,business badge holder,"Durable - Click-Fold Convex Name Badge Holder,...","Attractive, contemporary horizontal Click-Fold...",3,0.471,4
8872,28542,cuisinart coffee maker,Cuisinart DCC-1100 - coffee maker - white,This classically designed coffeemaker is accen...,4,0.000,2


In [19]:
for i in range(5, 10, 1):
    print 'Query:  %s ' %(mistakes.iloc[i].query)
    print 'Title: %s ' %(mistakes.iloc[i].product_title)
    print 'Description: %s ' %(mistakes.iloc[i].product_description)
    print '########################################'
    print 'Relevance %d ' %(mistakes.iloc[i].median_relevance)
    print 'My Prediction %d ' %(mistakes.iloc[i].pred)
    print '\n\n'

Query:  wireless mouse 
Title: Gear Head Mouse 
Description: Optical - Wireless - Radio Frequency - Black - 1000 dpi - Scroll Wheel 
########################################
Relevance 3 
My Prediction 4 



Query:  business badge holder 
Title: Advantus ID Badge Holder w/Convention Neck Pouch, Vertical, 2 1/4 x 3 1/2, Black, 12/PK 
Description: Badge ID holders feature a nylon convention pouch to display identification cards as well as offer additional storage. Back features two pockets: one open and one zippered to store credit cards or other items. Adjustable 48" cord. Insert Width: 2 1/4"; Insert Height: 3 1/2"; Orientation: Vertical; Color(s): Black.
Material: Nylon
Fob: Photo holder
Dimensions: 3.5 " x ; 2.25 "
Manufacturer Suggested Age: 3 Years and Up
CPSC Choking Hazard Warnings no_warning_applicabl 
########################################
Relevance 3 
My Prediction 4 



Query:  ps2 controller usb 
Title: Limited Edition Datel Rapid Fire Wildfire 2 Wired USB Game Pad Ruby Red

In [20]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [21]:
def query_terms_in_results(train):
    query_terms = train['query'].split(' ')
    response = train['product_title'] + ' ' + train['product_description']
    keyword = False
    
    for q in query_terms:
        if q not in stop:
            keyword = True
            if response.lower().find(q) == -1:
                return 0
    
    if keyword == False:
        return 0
    else:
        return 1
    
is_query_in_response = crowd_train.apply(query_terms_in_results, axis=1)

In [22]:
feature_query_in_response = pd.DataFrame({'is_query_in_response': is_query_in_response,
              'median_relevance': crowd_train.median_relevance})

In [23]:
feature_query_in_response.corr()

,is_query_in_response,median_relevance
is_query_in_response,1.00000,0.34641
median_relevance,0.34641,1.00000


In [24]:
feature_query_in_response.groupby(['is_query_in_response', 'median_relevance']).size()

is_query_in_response  median_relevance
0                     1                    613
                      2                    875
                      3                    774
                      4                   1629
1                     1                    161
                      2                    601
                      3                    963
                      4                   4542
dtype: int64

In [25]:
# lets find out how many query terms found in response
def count_query_terms_in_response(train):
    query_terms = train['query'].split(' ')
    response = train['product_title'] + ' ' + train['product_description']
    
    count = 0
    for q in query_terms:
        if response.lower().find(q) != -1:
            count += 1
    return count

num_query_terms_in_response = crowd_train.apply(count_query_terms_in_response, axis=1)

In [26]:
feature_num_query_in_response = pd.DataFrame({'num_query_in_response': num_query_terms_in_response,
              'median_relevance': crowd_train.median_relevance})

In [27]:
feature_num_query_in_response.corr()

,median_relevance,num_query_in_response
median_relevance,1.000000,0.250085
num_query_in_response,0.250085,1.000000


In [28]:
feature_num_query_in_response.groupby(['num_query_in_response', 'median_relevance']).size()

num_query_in_response  median_relevance
0                      1                    176
                       2                    148
                       3                    129
                       4                    210
1                      1                    323
                       2                    495
                       3                    512
                       4                   1295
2                      1                    232
                       2                    598
                       3                    711
                       4                   3144
3                      1                     37
                       2                    190
                       3                    304
                       4                   1076
4                      1                      6
                       2                     40
                       3                     72
                       4                    408


In [29]:
# lexical diversity
def word_diversity(train):
    response = train['product_title'] + ' ' + train['product_description']
    vocabulary = list(set([x.lower() for x in response.split(' ')]))
    
    return (len(vocabulary) * 1.) / len(response.split(' '))

word_diversity = crowd_train.apply(word_diversity, axis=1)

In [30]:
feature_word_diversity = pd.DataFrame({'word_diversity': word_diversity,
                                       'median_relevance': crowd_train.median_relevance})

In [31]:
feature_word_diversity.corr()

,median_relevance,word_diversity
median_relevance,1.000000,0.026877
word_diversity,0.026877,1.000000


In [32]:
# how big the response is
def response_length(train):
    response = train['product_title'] + ' ' + train['product_description']
    return len(response.split(' '))

query_response_length = crowd_train.apply(response_length, axis=1)

In [33]:
feature_response_length = pd.DataFrame({'response_length': query_response_length,
                                        'median_relevance': crowd_train.median_relevance})

In [34]:
feature_response_length.corr()

,median_relevance,response_length
median_relevance,1.000000,-0.022693
response_length,-0.022693,1.000000


### Stacked model with handcrafted features

In [56]:
%run query_features.py

In [57]:
svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

#keywords = keyword_counter(crowd_train.iloc[train_index])
keywords = keyword_counter(crowd_train)

In [58]:
features = stack([keywords, Xfitted])

In [59]:
from sklearn.pipeline import Pipeline

In [60]:
pipeline = Pipeline([('svd', svd), ('scl', scl), ('clf', clf)])

In [61]:
pipeline.fit(features, y)

Pipeline(steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
       random_state=None, tol=0.0)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [62]:
#keywords_test = keyword_counter(crowd_train.iloc[test_index])
keywords_test = keyword_counter(crowd_test)

In [63]:
Xtest = tfv.transform(Xv)

In [64]:
features_test = stack([keywords_test, Xtest])

In [65]:
preds_new_model = pipeline.predict(features_test)

In [66]:
make_submission(crowd_test.id.values.astype(int), preds_new_model, 'additionalfeatures.csv')

In [45]:
#print 'Kappa score on validation set %0.4f ' %(quadratic_weighted_kappa(yv, preds_new_model))

Kappa score on validation set 0.6520 


In [67]:
ensemble = (preds + preds_new_model) / 2

In [68]:
make_submission(crowd_test.id.values.astype(int), ensemble, 'ensemble300andadditional.csv')

In [46]:
#print 'Ensemble of the two model is %0.4f ' %(quadratic_weighted_kappa(yv, (preds + preds_new_model) / 2))

Ensemble of the two model is 0.6747 


In [69]:
best_score = pd.read_csv('./submissions/best_score.csv')

In [75]:
ensemble_best_score_additional = ((preds_new_model) + (best_score.prediction)) / 2

In [77]:
ensemble_best_score_additional_int = [int(x) for x in ensemble_best_score_additional]

In [78]:
ensemble_best_score_additional_int[:10]

[4, 3, 3, 2, 4, 4, 4, 3, 4, 3]

In [79]:
make_submission(crowd_test.id.values.astype(int), ensemble_best_score_additional_int, 'best_and_additional.csv')